In [1]:
import numpy as np
from pydrake.geometry import StartMeshcat
from pydrake.multibody.inverse_kinematics import (
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsStatus,
    DoDifferentialInverseKinematics,
)
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, EventStatus, LeafSystem
from pydrake.visualization import MeshcatPoseSliders

from manipulation import running_as_notebook
from manipulation.meshcat_utils import WsgButton
from manipulation.scenarios import AddIiwaDifferentialIK, ExtractBodyPose
from manipulation.station import MakeHardwareStation, load_scenario

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [3]:
def construct_scenario():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://tucker/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: panda
    file: package://drake/manipulation/models/franka_description/urdf/panda_arm.urdf
    default_joint_positions:
        panda_joint1: [0]
        panda_joint2: [0]
        panda_joint3: [0]
        panda_joint4: [0]
        panda_joint5: [0]
        panda_joint6: [0]
        panda_joint7: [1.57]
- add_weld:
    parent: table_top_link
    child: panda::panda_link0
    X_PC:
        translation: [0.35, 0.5, 0.015]
- add_model:
    name: panda_hand
    file: package://drake/manipulation/models/franka_description/urdf/panda_hand.urdf
- add_weld:
    parent: panda::panda_link7
    child: panda_hand::panda_hand
    X_PC:
        translation: [0, 0, 0.12]
        rotation: !Rpy { deg: [0, 0, 90]}
"""
# plant_config:
#    time_step: 0.01

    driver_data = """
model_drivers:
    panda: !IiwaDriver
      hand_model_name: panda_hand
    panda_hand: !SchunkWsgDriver {}
"""

    scenario_data += driver_data
    scenario = load_scenario(data = scenario_data)
    return scenario

In [4]:
from pydrake.all import (DifferentialInverseKinematicsIntegrator)

In [5]:

def AddPandaDifferentialIK(builder, plant, frame=None):
    params = DifferentialInverseKinematicsParameters(
        plant.num_positions(), plant.num_velocities()
    )
    time_step = plant.time_step()
    q0 = plant.GetPositions(plant.CreateDefaultContext())
    params.set_nominal_joint_position(q0)
    params.set_end_effector_angular_speed_limit(2)
    params.set_end_effector_translational_velocity_limits(
        [-2, -2, -2], [2, 2, 2]
    )
    if plant.num_positions() == 3:  # planar iiwa
        iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits)
        )
        # These constants are in body frame
        assert (
            frame.name() == "panda_link_7"
        ), "Still need to generalize the remaining planar diff IK params for different frames"  # noqa
        params.set_end_effector_velocity_flag(
            [True, False, False, True, False, True]
        )
    else:
        panda_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
        params.set_joint_velocity_limits(
            (-panda_velocity_limits, panda_velocity_limits)
        )
        params.set_joint_centering_gain(10 * np.eye(7))
    if frame is None:
        frame = plant.GetFrameByName("body")
    differential_ik = builder.AddSystem(
        DifferentialInverseKinematicsIntegrator(
            plant,
            frame,
            time_step,
            params,
            log_only_when_result_state_changes=True,
        )
    )
    return differential_ik

In [6]:
def teleop_3d():
    meshcat.ResetRenderMode()
    builder = DiagramBuilder()
    scenario = construct_scenario()
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml']))
    plant = station.GetSubsystemByName("plant")
    #station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    # TODO(russt): Replace with station.AddDiffIk(...)
    controller_plant = station.GetSubsystemByName(
        "panda.controller"
    ).get_multibody_plant_for_control()
    # Set up differential inverse kinematics.
    differential_ik = AddPandaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"),
    )
    builder.Connect(
        differential_ik.get_output_port(),
        station.GetInputPort("panda.position"),
    )
    builder.Connect(
        station.GetOutputPort("panda.state_estimated"),
        differential_ik.GetInputPort("robot_state"),
    )

    # Set up teleop widgets.
    meshcat.DeleteAddedControls()
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            lower_limit=[0, -0.5, -np.pi, -0.6, -0.8, 0.0],
            upper_limit=[2 * np.pi, np.pi, np.pi, 0.8, 0.3, 1.1],
        )
    )
    builder.Connect(
        teleop.get_output_port(), differential_ik.GetInputPort("X_WE_desired")
    )
    # Note: This is using "Cheat Ports". For it to work on hardware, we would
    # need to construct the initial pose from the HardwareStation outputs.
    plant = station.GetSubsystemByName("plant")
    ee_pose = builder.AddSystem(
        ExtractBodyPose(
            station.GetOutputPort("body_poses"),
            plant.GetBodyByName("panda_link7").index(),
        )
    )
    builder.Connect(
        station.GetOutputPort("body_poses"), ee_pose.get_input_port()
    )
    builder.Connect(ee_pose.get_output_port(), teleop.get_input_port())
    panda_hand_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(
        panda_hand_teleop.get_output_port(0), station.GetInputPort("panda_hand.position")
    )

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_context()
    simulator_context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(simulator_context)
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            print(simulator.get_context().get_continuous_state_vector())    
        meshcat.DeleteButton("Stop Simulation")

    else:
        simulator.AdvanceTo(0.1)


teleop_3d()

Press Space to open/close the gripper
Press Escape to stop the simulation
[0.05002777262733285, 0.06896460107142612, 0.028207119256898873, 0.18151016398838554, 0.0701021667024923, -0.3995816399479858, -0.3573921219039096]
[0.09536878807226008, 0.12944115522111294, 0.05419060100397764, 0.32103117412482035, 0.1336265300464568, -0.8372640240911619, -0.6838319452552348]


KeyboardInterrupt: 